<a href="https://colab.research.google.com/github/SatadruMukherjee/Data-Preprocessing-Models/blob/main/Snowflake_Postgres_semantic_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.snowflake.com/en/product/features/postgres/

In [1]:
!pip install psycopg2-binary

In [7]:
import psycopg2

conn = psycopg2.connect(
    host="",   # e.g. db.example.com
    port=5432,
    database="",
    user="",
    password="",
    sslmode="require"
)

cur = conn.cursor()
cur.execute("SELECT version();")
print(cur.fetchone())



('PostgreSQL 18.1 on aarch64-unknown-linux-gnu, compiled by gcc (GCC) 11.5.0 20240719 (Red Hat 11.5.0-11), 64-bit',)


In [8]:
cur.execute("CREATE EXTENSION vector;")

In [9]:
cur.execute("SELECT typname FROM pg_type WHERE typname = 'vector';")
cur.fetchone()

('vector',)

In [10]:
cur.execute("CREATE TABLE test_embeddings(product_id bigint, embeddings vector(3) );")
cur.execute("INSERT INTO test_embeddings VALUES (1, '[1, 2, 3]'), (2, '[2,-3,-4]');")


In [12]:
cur.execute("""SELECT product_id, embeddings, embeddings <=> '[3,1,2]' AS distance
FROM test_embeddings
ORDER BY embeddings <=> '[3,1,2]';""")
cur.fetchall()

[(1, '[1,2,3]', 0.2142857142857143), (2, '[2,-3,-4]', 1.2481458334927327)]

In [13]:
def cosine_distance(a,b):
  return 1-(a[0]*b[0]+a[1]*b[1]+a[2]*b[2])/(((a[0]**2+a[1]**2+a[2]**2)**0.5)*((b[0]**2+b[1]**2+b[2]**2))**0.5)

In [15]:
cosine_distance([1, 2, 3],[3,1,2])

0.2142857142857143

In [16]:
cosine_distance([2,-3,-4],[3,1,2])

1.2481458334927327

In [19]:
cur.execute("commit")

InterfaceError: cursor already closed

In [18]:
cur.close()
conn.close()